In [86]:
#!pip install holoviews # remove ! if you are not in Colab
#!pip install datashader[complete] # remove ! if you are not in Colab
#!pip install python-louvain # remove ! if you are not in Colab

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

# Introduction to Network Analysis

Welcome to your first part of the introduction to network analysis. In this session you will learn:

1. Why applying network analysis is helpful to answer certain questions, and why framing certain contexts as networks gives new insights.
2. The basic structure of relational data.
3. How to construct graph objects from different datasources.
4. How to analyse basic features of nodes, edges, and graphs.
5. How to identify groups and communities in graphs.
6. How to do simple network visualizations.

Generally, networks are a form of representing **relational data**. This is a very general tool that can be applied to many different types of relationships between all kind of elements. The content, meaning, and interpretation for sure depends on what elements we display, and which types of relationships. For example:

* In Social Network Analysis:
     * Nodes represent actors (which can be persons, firms and other socially constructed entities)
     * Edges represent relationships between this actors (friendship, interaction, co-affiliation, similarity ect.)
* Other types of network
     * Chemistry: Interaction between molecules
     * Computer Science: The wirld-wide-web, inter- and intranet topologies
     * Biology: Food-web, ant-hives

The possibilities to depict relational data are manifold. For example:

* Relations among persons
     * Kinship: mother of, wife of...
     * Other role based: boss of, supervisor of...
     * Affective: likes, trusts...
     * Interaction: give advice, talks to, retweets...
     * Affiliation: belong to same clubs, shares same interests...
* Relations among organizations
     * As corporate entities, joint ventures, strategic alliances
     * Buy from / sell to, leases to, outsources to
     * Owns shares of, subsidiary of
     * Via their members (Personnel flows, friendship...)

# Network data structures


## Edgelists
Most real world relational data is to be found in what we call an **edge list**, a dataframe that contains a minimum of two columns, one column of *nodes* that are the source of a connection and another column of nodes that are the target of the connection. The nodes in the data are identified by unique IDs.

If the distinction between source and target is meaningful, the network is **directed**. If the distinction is not meaningful, the network is **undirected** (more on that later). So, every row that contains the ID of one element in column 1, and the ID of another element in column 2 indicates that a connection between them exists.

An edge list can also contain additional columns that describe **attributes** of the edges such as a magnitude aspect for an edge. If the edges have a magnitude attribute the graph is considered **weighted** (e.g., number of interactions, strenght of friendship).


* Below an example of a minimal edge list created by zipping together two lists into a list of tuples.
* In this case, let us assume this network to be unweighted, meaning a connection can be eiter tresent or absent.


In [87]:
origin = [1, 2, 3, 1, 4, 5]
target = [2, 3, 4, 3, 5, 1]

In [ ]:
edge_list = list(zip(origin, target))

print(edge_list)



This can of cause also be a DataFrame where columns indicate origin and target

In [ ]:
df = pd.DataFrame({'origin':origin, 'target':target})
df


## Adjacency Matrix

* A second popular form of network representation is the **adjacency-matrix** (also called **socio-matrix**).
* It is represented as a $n*n$ matrix, where $n$ stands for the number of elements of which their relationships should be represented.
* The value in the cell that intercepts row $n$ and column $m$ indicates if an edge is present (=1) or absent (=0).
* Tip: Given an edgelist, an adjacency matrix can easily be produced by crosstabulating:



In [ ]:
mat_adj = pd.crosstab(df.origin, df.target)
mat_adj

In [91]:
from scipy import sparse

In [ ]:
# CSR Matrix - Compressed Sparse matrix - showing adjacency matrix without zeros
df_adj_sparse = sparse.csr_matrix(mat_adj)
print(df_adj_sparse)

## Nodelists
* Edgelists as well as adjacency matrices only stores connectivity pattern between nodes, but due to their structure cannot store informations on the nodes in which we might be interested.
* Therefore, we in many cases also provide a a **node list** with these informations (such as the names of the nodes or any kind of groupings).


In [93]:
name = ["Jesper", "Pernille", "Jacob", "Dorte", "Donald"]
sex = ["M", "F", "M", "F", "M"]
group = ["A", "B", "B", "A", "C"]

node_list = pd.DataFrame({'name':name, 'sex':sex, 'group':group})

In [ ]:
node_list

## Graph Objects (NetworkX)

Up to now we see that relatonal data, and the analysis thereof, has some particularities, making it distinct from tabular data (e.g., dataframes), we usually work with.

* Tabular data
     * In tabular data, summary statistics of variables are **between observations** (column-wise) interdependent, meaning changing a value of some observation will change the corresponding variables summary statistics.
     * LIkewise, variable values might be **within observation** interdependent (row-wise), meaning changing a variable value might change summary statistics of the observation
     * Otherwise, values are (at least mathematically) independent.
* Graph data
     * Same holds true, but adittional interdependencies due to the relational structure of the data.
     * Sepperation between **node** and **edge** data, which is interdependent. Removing a node might alos impy the removal of edges, removal of edges changes the characteristics of nodes
     * In adittion, the relational structure makes that not only true for adjacent nodes and edges, but potentially multiple. Adding/Removing one node/edge could change the characteristics of every single other node/edge.
     * That is less of a problem for local network characteristics (eg., a node's degree on level 1). However, many node and edge characteristics such
     * That's mainly why graph computing is slightly more messy, and need own mathematical tools, and applications from graphical computing (graphical like graph, not like figure)

A graph object is a specific datastructure which contains node and edgelists jointly, and enables the application of graph algorithms on them. We in the following will work with the [`networkx`](https://networkx.github.io/documentation/stable/index.html) library, which is the standard for network analysis in the Python community. Another popular package more from the graph-theorist community is [`igraph`](https://igraph.org/python/), which is also the main package used for network analysis in the `R` community. However, since `networkx` became the `Python` standard, we will in the following stick to it.

In [95]:
import networkx as nx # Main network analysis library

In NetworkX, graph data are stored in a dictionary-like fashion.
They are placed under a `Graph` object,
canonically instantiated with the variable `G` as follows:

```python
G = nx.Graph()
```

Of course, you are free to name the graph anything you want!

Nodes are part of the attribute `G.nodes`.
There, the node data are housed in a dictionary-like container,
where the key is the node itself
and the values are a dictionary of attributes.
Node data are accessible using syntax that looks like:

```python
G.nodes[node1]
```

Edges are part of the attribute `G.edges`,
which is also stored in a dictionary-like container.
Edge data are accessible using syntax that looks like:

```python
G.edges[node1, node2]
```
Because of the dictionary-like implementation of the graph,
any hashable object can be a node.
This means strings and tuples, but not lists and sets.

### Types of Graphs


1. Weigthed vs. Unweighted
2. Directed vs. Undirected
3. Unimodal vs. Multimodal
4. Unidimensional vs. Multidimensional

`networkx` graph classes
1. Graph
2. DiGraph
3. MultiGraph
4. MultiDigraph

### Creating a graph object

The graph object forms the core of network analysis. This is the central place where nodes, edges, and their characteristics are jointly stored. Lets take a look how to create one.

In [ ]:
# We can create a network graph G directly from the adjacency matrix.
G = nx.from_numpy_array(df_adj_sparse)
print(G.nodes(), G.edges())

In [ ]:
# We can also directly load an edgelist instead
G = nx.from_edgelist(edge_list)
print(G.nodes(), G.edges())

In [ ]:
# We can also create an own graph
G = nx.Graph() # create an empty graph with no nodes and no edges
print(G.nodes(), G.edges())

In [ ]:
G.add_nodes_from([1, 2, 3, 4, 5]) # add a list of nodes
G.add_edges_from([(1,2), (1,3), (4,5), (5,3), (4,2), (3,2), (1,4)]) # add a list of edges
print(G.nodes(), G.edges())

Ok, up to now that was pretty abstract, right? Lets plot the graph to get a feeling what the network looks like. We will explore this later more exhaustive.

In [ ]:
nx.draw(G, with_labels=True)


### Your turn!

So, now its already time for a little exercise: Try to create and plot the following network structure. Have fun :)

![](https://www.dropbox.com/s/88anz0w24kurckp/networks_mini_exercise.png?dl=1)

### Node Attributes


In [101]:
G = nx.from_edgelist([('Jesper', 'Pernille'), ('Jesper', 'Donald'), ('Jesper', 'Dorte'), ('Pernille', 'Jacob'), ('Pernille', 'Dorte')])

We can also include the node attributes in the graph object

In [ ]:
# Graph objects are dictionaries, so attributes need to be dictionaries too. They are matched via the index (=node name/id)
node_attr = node_list.set_index('name').to_dict('index')

node_attr

In [103]:
# set node attributes
nx.set_node_attributes(G, node_attr)

In [ ]:
# view nodes with attributes
G.nodes(data=True)

In [ ]:
G.edges()

In [ ]:
# First (not pretty) visualization
plt.figure(figsize=(5,5))
nx.draw_kamada_kawai(G, with_labels = True)

In [ ]:
# Node characteristics
G.nodes()['Jesper']

In [ ]:
# Node and adjacencies
G['Jesper']

In [ ]:
# Conditional node selection
node_list[node_list.sex == 'F'].name

In [110]:
# subsetting
G_sub = nx.subgraph(G, node_list[node_list.sex == 'F'].name)

# Network analysis and measures

## Network anlaysis of a novel


**Viktor Hugo's LES MISERABLES**


"Les Misérables" is a classic French novel written by Victor Hugo, first published in 1862. Set in the early 19th century, the story spans several decades and focuses on themes of justice, love, redemption, and social inequality. The central plot follows **Jean Valjean**, a former convict who is struggling to escape his past and lead an honest life after serving 19 years in prison for stealing a loaf of bread. Throughout the novel, Valjean is relentlessly pursued by **Javert**, a determined and morally rigid police inspector.

The book is a rich tapestry of interconnected characters, including **Fantine**, a destitute woman who leaves her daughter, **Cosette**, in Valjean's care; **Marius**, a young revolutionary in love with Cosette; and a group of rebellious students who rise up against the monarchy in the Paris Uprising of 1832. "Les Misérables" paints a powerful portrait of the human condition, focusing on social injustice, poverty, and the quest for moral and spiritual redemption amidst the backdrop of a politically and socially turbulent France.


<img src="https://m.media-amazon.com/images/I/81X4gUbRALL._AC_UF1000,1000_QL80_.jpg" alt="Image" width="300">




In [111]:
# lets read the graph from Networkx's package
G = nx.les_miserables_graph()

In [ ]:
# we print all the nodes (in our case characters) from the book
G.nodes()

In [ ]:
# a basic information about the graph
print(f"Graph info:")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"Graph density: {nx.density(G)}") # how close a graph is to being a complete graph
print(f"Graph is directed: {G.is_directed()}")
print(f"Node list: {list(G.nodes())[:10]}")  # Display first 10 nodes
print(f"Edge list: {list(G.edges())[:10]}")

Lets take a first visual peek. For a non-trivial (not like the one before) visualization of networks, the way how nodes and edges are placed in the plot makes a big difference in terms of how informative the visualization is, and how much information we can get out of it.

There are plenty of different algorithms to optimize node and edge positions in visualizations within a graph **layout**. [Here](https://networkx.github.io/documentation/stable/reference/drawing.html#module-networkx.drawing.layout) you find an overview over the ones implemented in `networkx`.

We can usually render these layouts on the fly when plotting the graph object. Since these algorithms contain probabilistic and iterative elements, they might look slightly different from plot to plot, though. Since we will plot this network a couple of times during this tutorial and want to compare them among each others, we will create and save the layout already upfront and pass it every time to the plotting function to make sure that the nodes are always placed in the same position.

Here, we use the *Fruchterman-Reingold* algorithm-one of the most commonly used ones-to create the layout.

In [114]:
# Create and save a layout.
G_layout = nx.layout.fruchterman_reingold_layout(G)

In [ ]:
# We use the standard networkx plot, and pass the layout.
nx.draw(G, pos = G_layout, with_labels=True)

Well, we somewhat an intuition of the network architecture, containing a core-region, and some periphral centers. Otherwise, it's rather uninformative. Good network visualization after all is more than just doing a standard plot, so there is much we can do to make it more informative. However, we get to that later.

In this case, we have a weighted graph, meaning that an edge does not only exist or not, (binary), but rather weighted by the number of the nodes (representing characters) interaction.

In [ ]:
nx.get_edge_attributes(G, 'weight')

We see that the edge weights vary quite a lot. For this simple example we will remove the weights to give us a undirected and unweighted graph.

In [117]:
for n1, n2, d in G.edges(data=True):
  d.pop('weight', None)

We can even do better! We can use the powerful [`bokeh`](https://docs.bokeh.org/) library (we installed it in the preamble). However, it is very clunky to work with. The [`hollowviews`](http://holoviews.org/user_guide/Network_Graphs.html) library in turn provides a lightweight wrapper that enables us to produce pretty interactive visualizations with a few lines of code. Lets do that!

PS: Try to hover over the nodes :-)

In [ ]:
# Import the libraries and link to the bokeh backend
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.plotting import show

# Setting the default figure size a bit larger
defaults = dict(width=750, height=750, padding=0.1,
                xaxis=None, yaxis=None)
hv.opts.defaults(
    opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

In [119]:
g_plot = hv.Graph.from_networkx(G, G_layout).opts(tools=['hover'])

In [ ]:
show(hv.render(g_plot))

## Node-Level measures

Often, we are interested in ways to summarize the pattern of node connectivity to infer something on their characteristics.


One of the simplest concepts when computing node level measures is that of **centrality**, i.e. how central is a node or edge in the graph. As this definition is inherently vague, a lot of different centrality scores exists that all treat the concept of "central" a bit different.

We in the following well briefly illustrate the idea behind three of the most popular centrality measures, namely:

* Degree centrality
* Eigenvector centrality
* Betweenness centrality

### Degree centrality
The degree centrality is probably the most intuitive node measure, which basically just counts the number of edges adjacent to a node.  Formally, the degree of node $i$ is the number of existing edges $e_{ij}$ with other nodes $j$ in a network with $n$ nodes:

$$d_{ij} =\sum\limits_{j=1}^{n} e_{ij} ~ where: ~ i \neq j$$

In [121]:
cent_degree = dict(nx.degree(G))

In [ ]:
cent_degree.values()

In [ ]:
sorted(cent_degree.items(),key=lambda x:-x[1])[:10]

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Gavroche_%28Les_Misérables%29.jpg/162px-Gavroche_%28Les_Misérables%29.jpg)

The most important character is *Valjean*, the main character, not that surprising. But the second most important character is *Gavroche*. Is that right? Lets have a look at the graph so we can see what is going on :-)

**Note**: Node size represents degree centrality - Bigger nodes indicate degree centrality.

In [ ]:
nx.set_node_attributes(G, cent_degree, 'cent_degree')

g_plot = hv.Graph.from_networkx(G, G_layout).opts(tools=['hover'],
                                                  node_size='cent_degree')

show(hv.render(g_plot))

In [ ]:
nx.get_node_attributes(G,'cent_degree')

### Eigenvector centrality
Similar to the degree centrality, the eigenvector centrality takes this idea of characterizing nodes by their importance in a network a step further. It also represents the main idea behind the pagerank algorithm (a variant of eigenvector centrality) that was powering Google Search in the beginning.

The basic idea is to weight a node's degree centrality by the centrality of the nodes adjacent to it (and their centrality in turn by their centrality). This will make nodes connected to in turn also well connected nodes more important. The eigenvector here is just a clever mathematical trick to solve such a recurrent problem.

$$x_{v}={\frac {1}{\lambda }}\sum _{t\in M(v)}x_{t}={\frac {1}{\lambda }}\sum _{t\in G}a_{v,t}x_{t}$$

In [126]:
cent_eigen = dict(nx.eigenvector_centrality(G))

In [ ]:
sorted(cent_eigen.items(),key=lambda x:-x[1])[:10]

In [128]:
for  i in cent_eigen:
  cent_eigen[i] = cent_eigen[i]*100

In [ ]:
nx.set_node_attributes(G, cent_eigen, 'cent_eigen')

g_plot = hv.Graph.from_networkx(G, G_layout).opts(tools=['hover'],
                                                  node_size='cent_eigen' )

show(hv.render(g_plot))

### Betweenness centrality

* The betweenness centrality of an object in a network measures the extent to which it lies on short paths
* A higher betweenness indicates that it lies on more short paths and hence should somehow be important for traversing between different parts of a network
* How many pairs of individuals would have to go through you in order to reach one another in the minimum number of hops? Who has higher betweenness, X or Y?

In formulaic representation

* The geodesic betweenness $B_{n}(i)$ of a **vertex** in a weighted, undirected network is
$$B_{n}(i) =  \sum_{s,t \in G} \frac{ \Psi_{s,t}(i) }{\Psi_{s,t}}$$
where vertices $s,t,i$ are all different from each other

* $\Psi_{s,t}$ denotes the number of shortest paths (geodesics) between vertices $s$ and $t$
* $\Psi_{s,t}(i)$ denotes the number of shortest paths (geodesics) between vertices $s$ and $t$ **that pass through vertex** $i$.
* The geodesic betweenness $B_n$ of a network is the mean of $B_n(i)$ over all vertices $i$

In [ ]:
cent_between = nx.betweenness_centrality(G)
sorted(cent_between.items(),key=lambda x:-x[1])[:10]

In [131]:
for  i in cent_between:
  cent_between[i] = cent_between[i]*100

In [ ]:
nx.set_node_attributes(G, cent_between, 'cent_between')

g_plot = hv.Graph.from_networkx(G, G_layout).opts(tools=['hover'],
                                                  node_size='cent_between' )

show(hv.render(g_plot))

## Neighborhood of a Node

Further, we can look at the surrounding of a node, meaning the ones it is connected to, its **neighborhood**. Here, we can look at the **ego-network of a node**. That means how many nodes are in a certain **geodesic distance**, meaning the **shortest path**. Plainly speaking, how many nodes are not more than x-steps away.

![](https://www.dropbox.com/s/yat7qsdfszmc1d1/networks_distance.jpg?dl=1)




In [ ]:
path = list(nx.all_shortest_paths(G, source="Valjean", target="Tholomyes"))
print(path)

In [ ]:
print(list(G.neighbors("Valjean")))

In [ ]:
len(list(G.neighbors("Valjean")))

In [136]:
def neighborhood(G, node, n):
    path_lengths = nx.single_source_dijkstra_path_length(G, node)
    return [node for node, length in path_lengths.items()
                    if length == n]

In [ ]:
print(neighborhood(G, 'Valjean', 2))

## Clustering (Community detection)

Another common operation is to group nodes based on the graph topology, sometimes referred to as *community detection* based on its commonality in social network analysis.

There are-just like for clustering of tabular data in UML-many different algorithms and approaches to detect and delineate communities. [Here](https://github.com/benedekrozemberczki/awesome-community-detection) you find a summary of currently used approaches.

The main logic in most cases is to find ways to form groups which have a maximum *within-connectivity* and a minimum *between-connectivity*. Consequently, nodes in the same community should have a higher probability of being connected than nodes from different communities.

Here we will use the **Louvain Method**, one of the most widely used community detection algorithms. It usually delivers good results, scales well, and can handle weighted networks. Furthermore, there is an actively maintained, easy to use Python implementation, [`python-louvain`](https://python-louvain.readthedocs.io).

It optimises a quantity called modularity:

$$  \sum_{ij} (A_{ij} - \lambda P_{ij}) \delta(c_i,c_j) $$

$A$ - The adjacency matrix

$P_{ij}$ - The expected connection between $i$ and $j$.

$\lambda$ - Resolution parameter

Can use lots of different forms for $P_{ij}$ but the standard one is the so called configuration model:

$P_{ij} = \frac{k_i k_j}{2m}$

Loosely speaking, in an iterative process it

1. You take a node and try to aggregate it to one of its neighbours.
2. You choose the neighbour that maximizes a modularity function. This function tells you how connected is the community you are trying to attach your node to going to be if you actually attach your node to it (this function is easy to compute and this makes Louvain very fast!).
3. Once you iterate through all the nodes, you will have merged few nodes together and formed some communities.
4. This becomes the new input for the algorithm that will treat each community as a node and try to merge them together to create bigger communities.
5. The algorithm stops when it's not possible to improve modularity any more.

![](https://www.dropbox.com/s/afoeq2pa01lba50/networks_louvain.jpg?dl=1)

This is the original paper, for those interested in further reads:

* Blondel, Vincent D; Guillaume, Jean-Loup; Lambiotte, Renaud; Lefebvre, Etienne (9 October 2008). "Fast unfolding of communities in large networks". Journal of Statistical Mechanics: Theory and Experiment. 2008 (10): P10008

Lets take a look how that works:


In [138]:
import community.community_louvain as community_louvain

In [139]:
# Find the optimal partition with the Louvain algorithm.
com = community_louvain.best_partition(G)

In [ ]:
# The number of communities detected
max(com.values())

In [ ]:
nx.set_node_attributes(G, com, 'community')

g_plot = hv.Graph.from_networkx(G, G_layout).opts(tools=['hover'],
                                                  node_size='cent_degree',
                                                  node_color='community', cmap=plt.cm.Set1,
                                                  legend_position='right')

show(hv.render(g_plot))

## (Global) Network structure

Finally, it is often also informative to look at the overal characteristics of the network.


The **density** of a measure represents the share of all connected to all possible connections in the network

In [ ]:
nx.density(G)

**Transistivity**, also called the **Clustering Cefficient** indicates how much the network tends to be locally clustered. That is measured by the share of **closed triplets**. Again,w e will dig into that next time.

![](https://www.dropbox.com/s/ei585dd6ysa243d/networks_ccoeff.png?dl=1)

In [ ]:
nx.transitivity(G)

The **diameter** is the longest of the shortest paths between two nodes of the network.

In [ ]:
nx.diameter(G)

Finally, the **mean distance**, or **average path lenght** represents the mean of all shortest paths between all nodes. It is a measure of diffusion potential within a network.

In [ ]:
nx.average_shortest_path_length(G)

One more thing we didn't talk about yet: Small worlds.

Small worlds are an interesting network structure, combining short path lenght betwen the nodes with a high clustering coefficient. That means, that we have small interconected clusters, which are in turn connected by **gatekeepers** (the edges we call **bridges** or **structural holes**).

# Case: Networks are coming...

![](https://sds-aau.github.io/SDS-master/00_media/random_got.jpg)

* So, lets get serious. Appropriate for the weather these days in Denmark, the theme is "winter is comming...".
* Therefore, we will have some fun analysing the Game of Thrones data provided by [Andrew Beveridge](https://github.com/mathbeveridge/asoiaf).
* It is a Character Interaction Networks for George R. R. Martin's "A Song of Ice and Fire" saga (yes, we are talking about the books...).
* These networks were created by connecting two characters whenever their names (or nicknames) appeared within 15 words of one another in one of the books in "A Song of Ice and Fire."
* The edge weight corresponds to the number of interactions.
* This is a nice skill you will have after the second part of M2 on your own.

## Build the graph

* First, we load all nodes, representing all characters appearing in the books:

In [146]:
edges = pd.read_csv('https://sds-aau.github.io/SDS-master/00_data/GoT_network/asoiaf-all-edges.csv')

In [ ]:
edges.head()

In [148]:
edges.columns = [w.lower() for w in edges.columns]

* So, that's what we have, a classical edgelist, with id1 in column 1 and id2 in column2.
* Note, the edges are in this case weighted.

Ok, lets see how many characters we have overall.



In [ ]:
pd.concat([edges['source'],edges['target']], axis=0).nunique()

In [ ]:
len(set(edges['source']) | set(edges['target']))


* Because there are so many characters in the books, many of them minor,
* I am subsetting the data to the 100 characters with the most interactions across all books.
* The edges are undirected, therefore there are no redundant Source-Target combinations.
* Because of this, I pivot Source and Target data before summing up the weights.

In [ ]:
edges_melt = pd.melt(edges, id_vars=['id','weight'], value_vars=['source', 'target'],
        var_name='role', value_name='name')

edges_melt.head()

In [ ]:
edges_melt.groupby('name').weight.sum().sort_values(ascending=False)

In [153]:
chars_main = edges_melt.groupby('name').weight.sum().sort_values(ascending=False).index[:100]


* So far so good, if we only go by edge weights,
* Lets reduce our edgelist to this main characters, just to warm up and keep the overview.



In [154]:
edges = edges[(edges.source.isin(chars_main)) & (edges.target.isin(chars_main))]

Now we can convert that into a NetowkrX graph

In [155]:
g = nx.from_pandas_edgelist(edges, source='source', target='target', edge_attr='weight')

In [ ]:
# no need to filter for parallel edges and seems there are no isolates (degree 0 nodes)
list(nx.isolates(g))

In [ ]:
g.edges(data=True)

In [ ]:
weights = [z['weight'] for x,y,z in g.edges(data=True)]
sns.displot(weights)


We see a right skewed distribution with many weak and some very strong edges. Lets take a look what are the edges with the highest weight (meaning here: the characters with most intraction).



In [159]:
centrality_dgr = nx.degree_centrality(g)
centrality_eigen = nx.eigenvector_centrality_numpy(g, weight='weight')
centrality_between = nx.betweenness_centrality(g, weight='weight')

In [160]:
nx.set_node_attributes(g, centrality_dgr, 'centrality_dgr')
nx.set_node_attributes(g, centrality_eigen, 'centrality_eigen')
nx.set_node_attributes(g, centrality_between, 'centrality_between')

In [ ]:
g.nodes(data=True)['Tyrion-Lannister']

In [ ]:
g.degree(weight='weight')['Tyrion-Lannister']


## Communities & Groups



In [163]:
partition = community_louvain.best_partition(g)

In [164]:
nx.set_node_attributes(g, partition, 'community')

In [165]:
nodes_df = pd.DataFrame.from_dict(dict(g.nodes(data=True)),orient='index')

In [ ]:
nodes_df.groupby('community')['centrality_eigen'].nlargest(5)


## Network Visualization I

Ok, lets give it a first minimal shot:



In [ ]:
nx.draw(g, with_labels = True, node_size=10)

In [168]:
# Create and save a layout.
g_layout = nx.layout.spring_layout(g)
g_plot = hv.Graph.from_networkx(g, g_layout).opts(tools=['hover'], node_color='community')
labels = hv.Labels(g_plot.nodes, ['x', 'y'], 'index')

In [169]:
from holoviews.operation.datashader import datashade, bundle_graph
bundled = bundle_graph(g_plot)

In [ ]:
show(hv.render(bundled * labels.opts(text_font_size='6pt', text_color='white', bgcolor='gray')))